In [1]:
import pandas as pd
import numpy as np
from statistics import mode
from scipy.stats import chi2_contingency,pearsonr
from fastparquet import ParquetFile
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import jaccard_score
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import torch

In [5]:
pf = ParquetFile("train_final.parquet")
pf_test = ParquetFile("test_final.parquet")

# Bir pandas dataFrame'ine dönüştürme
df_train = pf.to_pandas()
df_test = pf_test.to_pandas()

df_train

,id,month,n_seconds_1,n_seconds_2,n_seconds_3,carrier,devicebrand,feature_0,feature_1,feature_2,...,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target
0,5beefd4d2bf4a4767e0df8108,10,5245.571,981.182,205.948,VODAFONE TR,Apple,-1.197737,1.113360,-1.123334,...,-2.613336,-2.032903,2.645719,-1.023478,1.658986,-1.559406,-2.161336,30,58,"menu2, menu4, menu5"
1,867285b116c063d5a8482f5be,10,5184.876,557.650,487.587,TURKCELL,samsung,-2.336352,2.567766,-0.494908,...,-0.983938,-1.453756,-0.021547,-0.195770,2.775513,-0.318980,-4.291473,21,45,"menu7, menu8, menu4"
2,c82a7cbd2e00d9b66c06bcadc,10,3835.618,3275.128,43.806,TURK TELEKOM,Redmi,-2.561455,2.061736,-0.184511,...,-1.668703,-3.599403,1.673868,0.631790,1.293131,-2.230909,-2.383524,19,61,"menu2, menu8, menu4"
3,f2d2b25073ccc298eced86897,10,3532.544,154.509,64.724,TURKCELL,samsung,-2.529918,3.358050,-0.851366,...,-1.861418,-1.219658,1.863495,0.213170,1.029710,-1.142185,-4.466191,2,41,"menu6, menu2, menu1"
4,7818c92a58af0f2cb7c361738,10,3344.192,787.896,715.115,VODAFONE TR,samsung,-2.922361,2.096124,0.060796,...,-0.142903,-1.875545,1.024499,-0.186423,-0.061626,-1.462175,-2.371206,23,85,"menu6, menu2, menu8"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94044,2e54f32ced9fae6ef802ceaa2,12,44.397,43.425,41.678,O2 - DE,HUAWEI,-1.531534,2.596604,0.340233,...,-1.484154,-1.988186,0.044385,0.863442,0.761354,1.073711,-3.560019,29,68,"menu2, menu8, menu4"
94045,6aa4ff0f6cc5ef4c2980b2862,12,44.331,43.977,40.620,TURKCELL,samsung,-1.268987,2.300487,0.231711,...,-0.632773,-1.624605,2.406462,-0.340132,-0.893553,-2.061401,-1.908158,25,43,"menu9, menu2, menu5"
94046,fa842185a0edd210845b78308,12,44.142,43.591,41.736,VODAFONE TR,Apple,-1.950039,2.805681,0.438200,...,0.184886,-3.477337,0.449427,0.445361,-0.495204,-0.955097,-3.843092,4,25,"menu6, menu2, menu4"
94047,db8f55b8499f8d8c05148240e,12,43.963,43.350,40.862,TURK TELEKOM,Redmi,-2.389140,2.358281,0.683524,...,-0.186062,-2.329308,0.864857,1.284815,-0.100755,-1.056479,-1.452037,25,60,"menu6, menu8, menu4"


In [6]:
df_train.describe()

,month,n_seconds_1,n_seconds_2,n_seconds_3,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,...,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
count,94049.000000,94049.000000,94049.000000,94049.000000,94049.000000,94049.000000,94049.000000,94049.000000,94049.000000,94049.000000,...,94049.000000,94049.000000,94049.000000,94049.000000,94049.000000,94049.000000,94049.000000,94049.000000,94049.000000,94049.000000
mean,11.311561,432.781818,197.904016,102.381759,-1.824932,1.948002,-0.014231,-0.270988,3.428983,2.314612,...,1.055930,-1.247314,-1.863433,1.042226,0.260601,0.737817,-0.683078,-2.333848,20.531978,50.843943
std,0.732301,429.834018,169.474603,80.533643,0.706014,0.742521,0.855615,1.001878,0.886168,0.765429,...,0.745859,0.767093,0.812613,0.910212,0.797621,0.837250,0.858325,0.642435,8.310330,12.950021
min,10.000000,41.850000,40.231000,40.001000,-5.009604,-1.190684,-4.988702,-4.205721,-2.057158,-1.572397,...,-3.565546,-4.956211,-6.151754,-3.265827,-3.735571,-3.631153,-4.166801,-5.680349,1.000000,20.000000
25%,11.000000,181.540000,91.886000,53.746000,-2.290048,1.443627,-0.560076,-0.950880,2.891524,1.805311,...,0.587355,-1.736227,-2.396253,0.431905,-0.252991,0.161747,-1.279131,-2.753271,15.000000,41.000000
50%,11.000000,306.947000,144.874000,75.415000,-1.840876,1.934019,0.005703,-0.337732,3.503459,2.316267,...,1.093342,-1.219592,-1.865288,1.037332,0.285148,0.703326,-0.703813,-2.327645,21.000000,50.000000
75%,12.000000,537.064000,241.504000,119.032000,-1.379623,2.431059,0.559322,0.330430,4.050990,2.825116,...,1.565953,-0.732870,-1.333491,1.648791,0.798862,1.276491,-0.112671,-1.909460,26.000000,60.000000
max,12.000000,13567.046000,3275.128000,1476.077000,2.137354,6.574112,4.055067,5.021944,6.272530,5.775909,...,3.869422,2.830491,3.225738,5.322931,3.825793,5.147134,3.483374,1.401900,68.000000,102.000000


In [7]:
df_train.month.value_counts()

month
12    44433
11    34485
10    15131
Name: count, dtype: int64

In [8]:
df_test.month.value_counts()

month
1    11955
Name: count, dtype: int64

In [9]:
df_train.devicebrand.value_counts()

devicebrand
Apple         33238
samsung       31971
Redmi          7709
HUAWEI         6595
xiaomi         5300
              ...  
Cat               1
HOMETECH          1
WIKO              1
Gigaset           1
blackshark        1
Name: count, Length: 64, dtype: int64

In [10]:
df_train.target.value_counts()[df_train.target.value_counts()>2000]

target
menu6, menu2, menu8    13583
menu6, menu2, menu4    12827
menu6, menu2, menu1     8891
menu6, menu2, menu5     4861
menu2, menu4, menu5     3861
menu6, menu2, menu9     3860
menu9, menu2, menu4     3525
menu9, menu2, menu6     2792
menu2, menu8, menu4     2321
menu2, menu4, menu8     2251
Name: count, dtype: int64

In [11]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

In [12]:
X = df_train.drop(columns=['id','carrier','devicebrand','target'])
print("Top Absolute Correlations")
top_abs_corr = get_top_abs_correlations(X, 5)
print(top_abs_corr)

Top Absolute Correlations


n_seconds_2  n_seconds_3    0.618358
feature_48   feature_49     0.590486
n_seconds_1  n_seconds_2    0.583850
feature_20   feature_21     0.581451
feature_9    feature_21     0.560666
dtype: float64


In [13]:
top_abs_corr = get_top_abs_correlations(X, 7)
print(top_abs_corr)

n_seconds_2  n_seconds_3    0.618358
feature_48   feature_49     0.590486
n_seconds_1  n_seconds_2    0.583850
feature_20   feature_21     0.581451
feature_9    feature_21     0.560666
feature_11   feature_44     0.541293
feature_8    feature_34     0.520911
dtype: float64


In [14]:
from sklearn.preprocessing import MinMaxScaler

df_train['corr_48_49'] = df_train['feature_48'] + df_train['feature_49']
df_test['corr_48_49'] = df_test['feature_48'] + df_test['feature_49']

# Min-Max ölçeklendirme nesnesini oluşturun
scaler48 = MinMaxScaler(feature_range=(-5, 5))
scaler49 = MinMaxScaler(feature_range=(-5, 5))
scalercorr = MinMaxScaler(feature_range=(-5, 5))

df_train['feature_48'] = scaler48.fit_transform(df_train[['feature_48']])
df_train['feature_49'] = scaler49.fit_transform(df_train[['feature_49']])
df_train['corr_48_49'] = scalercorr.fit_transform(df_train[['corr_48_49']])
df_test['feature_48'] = scaler48.transform(df_test[['feature_48']])
df_test['feature_49'] = scaler49.transform(df_test[['feature_49']])
df_test['corr_48_49'] = scalercorr.transform(df_test[['corr_48_49']])

#df_train['corr_48_49'] = df_train['feature_48'] + df_train['feature_49']
#df_test['corr_48_49'] = df_test['feature_48'] + df_test['feature_49']

In [15]:
"""
for i in top_abs_corr.index:
    label = i[0] + i[1]
    df_train[label] = df_train[i[0]] + df_train[i[1]]
    df_test[label] = df_test[i[0]] + df_test[i[1]]
"""

'\nfor i in top_abs_corr.index:\n    label = i[0] + i[1]\n    df_train[label] = df_train[i[0]] + df_train[i[1]]\n    df_test[label] = df_test[i[0]] + df_test[i[1]]\n'

In [16]:
len(df_train.feature_48.unique()),len(df_train.feature_49.unique())

(61, 83)

In [17]:
len(df_test.feature_48.unique()),len(df_test.feature_49.unique())

(55, 79)

In [18]:
print(df_train.groupby("feature_48")["target"].apply(lambda x: mode(x)))

feature_48
-5.000000    menu6, menu2, menu4
-4.850746    menu6, menu2, menu4
-4.701493    menu6, menu2, menu4
-4.552239    menu6, menu2, menu4
-4.402985    menu6, menu2, menu8
                    ...         
 3.358209    menu6, menu2, menu4
 3.656716    menu2, menu8, menu4
 3.955224    menu6, menu2, menu9
 4.552239    menu9, menu4, menu1
 5.000000    menu6, menu2, menu4
Name: target, Length: 61, dtype: object


In [19]:
"""
fe48_mode=pd.DataFrame(df_train.groupby("feature_48")["target"].apply(lambda x: mode(x)))
df_train=pd.merge(df_train,fe48_mode,how="inner",on="feature_48").rename({"target_x":"target","target_y":"fe48_mode"},axis=1)
df_test=pd.merge(df_test,fe48_mode,how="inner",on="feature_48").rename({"target":"fe48_mode"},axis=1)
"""

'\nfe48_mode=pd.DataFrame(df_train.groupby("feature_48")["target"].apply(lambda x: mode(x)))\ndf_train=pd.merge(df_train,fe48_mode,how="inner",on="feature_48").rename({"target_x":"target","target_y":"fe48_mode"},axis=1)\ndf_test=pd.merge(df_test,fe48_mode,how="inner",on="feature_48").rename({"target":"fe48_mode"},axis=1)\n'

In [20]:
df_train

,id,month,n_seconds_1,n_seconds_2,n_seconds_3,carrier,devicebrand,feature_0,feature_1,feature_2,...,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target,corr_48_49
0,5beefd4d2bf4a4767e0df8108,10,5245.571,981.182,205.948,VODAFONE TR,Apple,-1.197737,1.113360,-1.123334,...,-2.032903,2.645719,-1.023478,1.658986,-1.559406,-2.161336,-0.671642,-3.658537e-01,"menu2, menu4, menu5",0.193798
1,867285b116c063d5a8482f5be,10,5184.876,557.650,487.587,TURKCELL,samsung,-2.336352,2.567766,-0.494908,...,-1.453756,-0.021547,-0.195770,2.775513,-0.318980,-4.291473,-2.014925,-1.951220e+00,"menu7, menu8, menu4",-1.511628
2,c82a7cbd2e00d9b66c06bcadc,10,3835.618,3275.128,43.806,TURK TELEKOM,Redmi,-2.561455,2.061736,-0.184511,...,-3.599403,1.673868,0.631790,1.293131,-2.230909,-2.383524,-2.313433,-8.881784e-16,"menu2, menu8, menu4",-0.426357
3,f2d2b25073ccc298eced86897,10,3532.544,154.509,64.724,TURKCELL,samsung,-2.529918,3.358050,-0.851366,...,-1.219658,1.863495,0.213170,1.029710,-1.142185,-4.466191,-4.850746,-2.439024e+00,"menu6, menu2, menu1",-3.294574
4,7818c92a58af0f2cb7c361738,10,3344.192,787.896,715.115,VODAFONE TR,samsung,-2.922361,2.096124,0.060796,...,-1.875545,1.024499,-0.186423,-0.061626,-1.462175,-2.371206,-1.716418,2.926829e+00,"menu6, menu2, menu8",1.744186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94044,2e54f32ced9fae6ef802ceaa2,12,44.397,43.425,41.678,O2 - DE,HUAWEI,-1.531534,2.596604,0.340233,...,-1.988186,0.044385,0.863442,0.761354,1.073711,-3.560019,-0.820896,8.536585e-01,"menu2, menu8, menu4",0.891473
94045,6aa4ff0f6cc5ef4c2980b2862,12,44.331,43.977,40.620,TURKCELL,samsung,-1.268987,2.300487,0.231711,...,-1.624605,2.406462,-0.340132,-0.893553,-2.061401,-1.908158,-1.417910,-2.195122e+00,"menu9, menu2, menu5",-1.356589
94046,fa842185a0edd210845b78308,12,44.142,43.591,41.736,VODAFONE TR,Apple,-1.950039,2.805681,0.438200,...,-3.477337,0.449427,0.445361,-0.495204,-0.955097,-3.843092,-4.552239,-4.390244e+00,"menu6, menu2, menu4",-4.379845
94047,db8f55b8499f8d8c05148240e,12,43.963,43.350,40.862,TURK TELEKOM,Redmi,-2.389140,2.358281,0.683524,...,-2.329308,0.864857,1.284815,-0.100755,-1.056479,-1.452037,-1.417910,-1.219512e-01,"menu6, menu8, menu4",-0.038760


In [21]:
target_column = 'target'
numerical_columns = df_train.select_dtypes(exclude=['object']).columns.tolist()
categorical_columns = df_train.select_dtypes(include=['object']).columns.tolist()

In [ ]:
for feature_column in numerical_columns:
    contingency_table = pd.crosstab(df_train['month'], df_train[feature_column])
    chi2, p, _, _ = chi2_contingency(contingency_table)

    # Elde edilen p-değeri ile ilişkinin anlamlılığını değerlendirin
    if p < 0.05:
        print(f"{feature_column} month ile anlamlı bir ilişkiye (p = {p}) sahip.")
    else:
        print(f"{feature_column} month hedef ile anlamlı bir ilişkiye (p = {p}) sahip değil.")

month month ile anlamlı bir ilişkiye (p = 0.0) sahip.
n_seconds_1 month hedef ile anlamlı bir ilişkiye (p = 0.7795833910796558) sahip değil.
n_seconds_2 month hedef ile anlamlı bir ilişkiye (p = 0.8971666007222011) sahip değil.
n_seconds_3 month hedef ile anlamlı bir ilişkiye (p = 0.9698473720160965) sahip değil.
feature_0 month hedef ile anlamlı bir ilişkiye (p = 0.4982655114823216) sahip değil.
feature_1 month hedef ile anlamlı bir ilişkiye (p = 0.4982655114823595) sahip değil.
feature_2 month hedef ile anlamlı bir ilişkiye (p = 0.49826551148234055) sahip değil.
feature_3 month hedef ile anlamlı bir ilişkiye (p = 0.49826551148234055) sahip değil.
feature_4 month hedef ile anlamlı bir ilişkiye (p = 0.4982655114823784) sahip değil.
feature_5 month hedef ile anlamlı bir ilişkiye (p = 0.4982655114823595) sahip değil.
feature_6 month hedef ile anlamlı bir ilişkiye (p = 0.49826551148234055) sahip değil.
feature_7 month hedef ile anlamlı bir ilişkiye (p = 0.4982655114823595) sahip değil.
fe

In [25]:
df_corr = df_train.drop(columns=['id','carrier','devicebrand','target'])

In [32]:
correlation_matrix = df_corr.corr()
neg_corr = correlation_matrix.where(np.triu(correlation_matrix < 0, k=1))
top_negative_corr = neg_corr.unstack().sort_values().dropna()
top_5_negative_corr = top_negative_corr.head(5)
print(top_5_negative_corr)

feature_21  feature_20   -0.581451
            feature_9    -0.560666
feature_44  feature_11   -0.541293
feature_34  feature_20   -0.503271
feature_44  feature_8    -0.491365
dtype: float64


In [33]:
"""liste1=df_train.carrier.value_counts()[df_train.carrier.value_counts()>4].index
liste2=df_test.carrier.value_counts()[df_test.carrier.value_counts()>0].index
fark1 = [eleman for eleman in liste1 if eleman not in liste2]
fark2 = [eleman for eleman in liste2 if eleman not in liste1]
print("liste1'de liste2'de olmayanlar:", fark1)
print("liste2'de liste1'de olmayanlar:", fark2)

print(f"liste1'de liste2'de olmayanların uzunluğu {len(fark1)}")
print(f"liste2'de liste1'de olmayanların uzunluğu {len(fark2)}")
df_train.loc[df_train.carrier.isin(fark1),'carrier'] = 'other'
df_test.loc[df_test.carrier.isin(fark2),'carrier'] = 'other'"""

'liste1=df_train.carrier.value_counts()[df_train.carrier.value_counts()>4].index\nliste2=df_test.carrier.value_counts()[df_test.carrier.value_counts()>0].index\nfark1 = [eleman for eleman in liste1 if eleman not in liste2]\nfark2 = [eleman for eleman in liste2 if eleman not in liste1]\nprint("liste1\'de liste2\'de olmayanlar:", fark1)\nprint("liste2\'de liste1\'de olmayanlar:", fark2)\n\nprint(f"liste1\'de liste2\'de olmayanların uzunluğu {len(fark1)}")\nprint(f"liste2\'de liste1\'de olmayanların uzunluğu {len(fark2)}")\ndf_train.loc[df_train.carrier.isin(fark1),\'carrier\'] = \'other\'\ndf_test.loc[df_test.carrier.isin(fark2),\'carrier\'] = \'other\''

In [34]:
"""
df_train[df_train['carrier'].isin(fark1)]
df_test[df_test['carrier'].isin(fark2)]
"""

"\ndf_train[df_train['carrier'].isin(fark1)]\ndf_test[df_test['carrier'].isin(fark2)]\n"

In [19]:
df_train.carrier.value_counts()[df_train.carrier.value_counts()>200]

carrier
TURKCELL         39043
VODAFONE TR      25583
TURK TELEKOM     22091
BIMCELL           1108
Unknown            684
KKTCELL            659
VODAFONE           560
HAYATEVESIGAR      559
TEKNOSACELL        498
PTTCELL            423
Name: count, dtype: int64

In [20]:
df_train.devicebrand.value_counts()[df_train.devicebrand.value_counts()>200]

devicebrand
Apple      33238
samsung    31971
Redmi       7709
HUAWEI      6595
xiaomi      5300
OPPO        3247
POCO         989
GM           770
realme       562
lge          442
TECNO        296
Casper       278
HONOR        264
Vestel       261
vivo         239
reeder       220
Sony         216
Name: count, dtype: int64

In [21]:
carrier_counts = df_train['carrier'].value_counts()
# Frekansı 400'den küçük olanları "low" olarak işaretleyin
low_carrier_mask = carrier_counts <= 400#400-20
low_carriers = carrier_counts[low_carrier_mask].index.tolist()
df_train.loc[df_train['carrier'].isin(low_carriers), 'carrier'] = 'other'


carrier_counts = df_test['carrier'].value_counts()
# Frekansı 400'den küçük olanları "low" olarak işaretleyin
low_carrier_mask = carrier_counts < 50#50-3
low_carriers = carrier_counts[low_carrier_mask].index.tolist()
df_test.loc[df_test['carrier'].isin(low_carriers), 'carrier'] = 'other'


In [22]:
df_train['carrier'] = df_train['carrier'].str.replace(' ', '_')
#df_train['carrier'] = df_train['carrier'].str.replace('TR', '')

df_test['carrier'] = df_test['carrier'].str.replace(' ', '_')
#df_test['carrier'] = df_test['carrier'].str.replace('TR', '')

In [23]:
"""
liste1=df_train.devicebrand.value_counts()[df_train.devicebrand.value_counts()>3].index
liste2=df_test.devicebrand.value_counts()[df_test.devicebrand.value_counts()>0].index
fark1 = [eleman for eleman in liste1 if eleman not in liste2]
fark2 = [eleman for eleman in liste2 if eleman not in liste1]
print("liste1'de liste2'de olmayanlar:", fark1)
print("liste2'de liste1'de olmayanlar:", fark2)

print(f"liste1'de liste2'de olmayanların uzunluğu {len(fark1)}")
print(f"liste2'de liste1'de olmayanların uzunluğu {len(fark2)}")

df_train.loc[df_train.devicebrand.isin(fark1),'devicebrand'] = 'other'
df_test.loc[df_test.devicebrand.isin(fark2),'devicebrand'] = 'other
'"""

'\nliste1=df_train.devicebrand.value_counts()[df_train.devicebrand.value_counts()>3].index\nliste2=df_test.devicebrand.value_counts()[df_test.devicebrand.value_counts()>0].index\nfark1 = [eleman for eleman in liste1 if eleman not in liste2]\nfark2 = [eleman for eleman in liste2 if eleman not in liste1]\nprint("liste1\'de liste2\'de olmayanlar:", fark1)\nprint("liste2\'de liste1\'de olmayanlar:", fark2)\n\nprint(f"liste1\'de liste2\'de olmayanların uzunluğu {len(fark1)}")\nprint(f"liste2\'de liste1\'de olmayanların uzunluğu {len(fark2)}")\n\ndf_train.loc[df_train.devicebrand.isin(fark1),\'devicebrand\'] = \'other\'\ndf_test.loc[df_test.devicebrand.isin(fark2),\'devicebrand\'] = \'other\n\''

In [24]:
devicebrand_counts = df_train['devicebrand'].value_counts()
# Frekansı 400'den küçük olanları "low" olarak işaretleyin
devicebrand_mask = devicebrand_counts <= 400#400
devicebrand = devicebrand_counts[devicebrand_mask].index.tolist()
df_train.loc[df_train['devicebrand'].isin(devicebrand), 'devicebrand'] = 'other'

devicebrand_counts = df_test['devicebrand'].value_counts()
# Frekansı 400'den küçük olanları "low" olarak işaretleyin
devicebrand_mask = devicebrand_counts < 50#50
devicebrand = devicebrand_counts[devicebrand_mask].index.tolist()
df_test.loc[df_test['devicebrand'].isin(devicebrand), 'devicebrand'] = 'other'

In [25]:
len(df_train.carrier.unique()),len(df_train.devicebrand.unique())

(11, 11)

In [26]:
len(df_test.carrier.unique()),len(df_test.devicebrand.unique())

(11, 11)

In [27]:
"""def get_binary_target(menu_str):
    
    # Virgülle ayrılmış menüleri bir liste olarak elde etmek için split() kullanabiliriz.
    menu_list = menu_str.split(",")
    #print(menu_list)
    
    # Sadece sayıları içeren bir liste oluşturmak için menü isimlerinden "menü" kısmını kaldıralım.
    numbers_list = [menu.strip().replace("menu", "") for menu in menu_list]

    # Elde edilen sayılar listesi
    #print(numbers_list)

    binary_list = [format(int(number), '04b') for number in numbers_list]

    # Elde edilen binary sayılar listesi
    #print(binary_list)

    binary_string = ''.join(binary_list)

    # Elde edilen binary dize
    #print(binary_string) 
    return binary_string

df_train['encoded_target'] = df_train['target'].apply(get_binary_target)"""

'def get_binary_target(menu_str):\n    \n    # Virgülle ayrılmış menüleri bir liste olarak elde etmek için split() kullanabiliriz.\n    menu_list = menu_str.split(",")\n    #print(menu_list)\n    \n    # Sadece sayıları içeren bir liste oluşturmak için menü isimlerinden "menü" kısmını kaldıralım.\n    numbers_list = [menu.strip().replace("menu", "") for menu in menu_list]\n\n    # Elde edilen sayılar listesi\n    #print(numbers_list)\n\n    binary_list = [format(int(number), \'04b\') for number in numbers_list]\n\n    # Elde edilen binary sayılar listesi\n    #print(binary_list)\n\n    binary_string = \'\'.join(binary_list)\n\n    # Elde edilen binary dize\n    #print(binary_string) \n    return binary_string\n\ndf_train[\'encoded_target\'] = df_train[\'target\'].apply(get_binary_target)'

In [28]:
def get_encoded_target(menu_str):
    menu_list = [menu.strip() for menu in menu_str.split(",")]
    ikili_dize = ""
    
    for menu in range(1, 10):  # Menu numaralarını 1'den 8'e kadar alın
            if f'menu{menu}' in menu_list:
                ikili_dize += "1"
            else:
                ikili_dize += "0"
            
    return ikili_dize
        
df_train['encoded_target'] = df_train['target'].apply(get_encoded_target)

In [29]:
len(df_train.target.unique()),len(df_train.encoded_target.unique())

(112, 84)

In [30]:
ohe = OneHotEncoder(drop='first',handle_unknown='ignore',sparse=False)
#ohe.fit(df_train[['carrier', 'devicebrand','fe48_mode']])
encoded_data_train  = ohe.fit_transform(df_train[['carrier', 'devicebrand']])#,'fe48_mode'
ohe_df = pd.DataFrame(encoded_data_train, columns=ohe.get_feature_names_out())
df_train = pd.concat([df_train, ohe_df], axis=1).drop(['carrier', 'devicebrand'], axis=1)#,'fe48_mode'

encoded_data_test = ohe.transform(df_test[['carrier', 'devicebrand']])#,'fe48_mode'
ohe_df = pd.DataFrame(encoded_data_test, columns=ohe.get_feature_names_out())
df_test = pd.concat([df_test, ohe_df], axis=1).drop(['carrier', 'devicebrand'], axis=1)#,'fe48_mode'

In [40]:
for feature_column in numerical_columns:
    contingency_table = pd.crosstab(df_train['encoded_target'], df_train[feature_column])
    chi2, p, _, _ = chi2_contingency(contingency_table)

    # Elde edilen p-değeri ile ilişkinin anlamlılığını değerlendirin
    if p < 0.05:
        print(f"{feature_column} kategorik hedef ile anlamlı bir ilişkiye (p = {p}) sahip.")
    else:
        print(f"{feature_column} kategorik hedef ile anlamlı bir ilişkiye (p = {p}) sahip değil.")

month kategorik hedef ile anlamlı bir ilişkiye (p = 0.0) sahip.
n_seconds_1 kategorik hedef ile anlamlı bir ilişkiye (p = 0.9999999994725743) sahip değil.
n_seconds_2 kategorik hedef ile anlamlı bir ilişkiye (p = 0.9999740725379133) sahip değil.
n_seconds_3 kategorik hedef ile anlamlı bir ilişkiye (p = 1.0) sahip değil.
feature_0 kategorik hedef ile anlamlı bir ilişkiye (p = 0.49155305390820936) sahip değil.
feature_1 kategorik hedef ile anlamlı bir ilişkiye (p = 0.49155305390661114) sahip değil.
feature_2 kategorik hedef ile anlamlı bir ilişkiye (p = 0.4915530539071752) sahip değil.
feature_3 kategorik hedef ile anlamlı bir ilişkiye (p = 0.4915530539067051) sahip değil.
feature_4 kategorik hedef ile anlamlı bir ilişkiye (p = 0.49155305390698717) sahip değil.
feature_5 kategorik hedef ile anlamlı bir ilişkiye (p = 0.4915530539073632) sahip değil.
feature_6 kategorik hedef ile anlamlı bir ilişkiye (p = 0.4915530539077393) sahip değil.


KeyboardInterrupt: 

In [31]:
# Bağımsız değişkenleri ve hedef değişkeni ayırın
X = df_train.drop(['id',"target",'encoded_target'], axis=1)#,'month'
y = df_train["encoded_target"] #df_train["target"]

In [32]:
from sklearn.preprocessing import LabelEncoder

# Hedef değişkeni etiket kodlama ile dönüştürün
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X,y)

pred_lr = model.predict(X)

c:\Users\necme\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
jaccard_similarity = jaccard_score(y,pred_lr,average='micro')

# Sonucu görüntüleyin
print("Jaccard Benzerliği:", jaccard_similarity)

Jaccard Benzerliği: 0.09466859879765584


In [33]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(n_estimators=100)

In [34]:
#upwork
xgb_model.fit(X,y_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
xgb_pred = xgb_model.predict(X)
xgb_pred = encoder.inverse_transform(xgb_pred)
xgb_pred

array(['001001000101', '011110000100', '001010000100', ...,
       '011000100100', '011010000100', '011000101000'], dtype=object)

In [ ]:
print(cross_val_score(xgb_model,X,y_encoded,cv=5,scoring='jaccard')) 
#200 [0.19138756 0.15901116 0.19707602 0.18856991 0.17970121]
#100 [0.18559277 0.15268474 0.19356725 0.18405104 0.17603275]

KeyboardInterrupt: 

In [ ]:
jaccard_similarity = jaccard_score(y,xgb_pred,average='micro')

# Sonucu görüntüleyin
print("Jaccard Benzerliği:", jaccard_similarity)


Jaccard Benzerliği: 0.6311809494077042


In [ ]:
jaccard_similarity = jaccard_score(y,xgb_pred,average='micro')

# Sonucu görüntüleyin
print("Jaccard Benzerliği:", jaccard_similarity)
#default 0.6383844190685237 with month
#Jaccard Benzerliği: 0.6319734855715005 400 50

Jaccard Benzerliği: 0.6383844190685237


In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# GridSearchCV ile modeli eğitim verilerine uyarlayın
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=3)
grid_search.fit(X, y_encoded)

# En iyi modeli bulun
best_model = grid_search.best_estimator_

# Test verilerini kullanarak modelin performansını değerlendirin
y_pred = best_model.predict(X)
y_pred = encoder.inverse_transform(y_pred)
jaccard_similarity = jaccard_score(y,y_pred,average='micro')

# Sonucu görüntüleyin
print("Jaccard Benzerliği:", jaccard_similarity)

# En iyi modelin hiperparametrelerini görüntüleyin
print("En iyi modelin hiperparametreleri:", grid_search.best_params_)


In [ ]:
cat_model=CatBoostClassifier()
cat_model.fit(X,y_encoded)

Learning rate set to 0.099762
0:	learn: 4.2076237	total: 6.16s	remaining: 1h 42m 33s
1:	learn: 4.0236389	total: 10.7s	remaining: 1h 28m 38s
2:	learn: 3.8932617	total: 14.9s	remaining: 1h 22m 40s
3:	learn: 3.7959093	total: 19.4s	remaining: 1h 20m 23s
4:	learn: 3.7179101	total: 23.9s	remaining: 1h 19m 23s
5:	learn: 3.6567390	total: 28.5s	remaining: 1h 18m 47s
6:	learn: 3.6038140	total: 33.6s	remaining: 1h 19m 21s
7:	learn: 3.5576521	total: 38.2s	remaining: 1h 18m 57s
8:	learn: 3.5173270	total: 43.1s	remaining: 1h 19m 2s
9:	learn: 3.4840928	total: 48s	remaining: 1h 19m 14s
10:	learn: 3.4554564	total: 52.4s	remaining: 1h 18m 32s
11:	learn: 3.4324191	total: 57s	remaining: 1h 18m 14s
12:	learn: 3.4092089	total: 1m 2s	remaining: 1h 18m 51s
13:	learn: 3.3880638	total: 1m 7s	remaining: 1h 19m 9s
14:	learn: 3.3698471	total: 1m 12s	remaining: 1h 19m 36s
15:	learn: 3.3542599	total: 1m 17s	remaining: 1h 19m 50s
16:	learn: 3.3393203	total: 1m 23s	remaining: 1h 20m 16s
17:	learn: 3.3251913	total: 1m 

KeyboardInterrupt: 

In [46]:
#X.columns = X.columns.str.replace('[^a-zA-Z0-9_]', '_', regex=True)
df_test.columns = df_test.columns.str.replace('[^a-zA-Z0-9_]', '_', regex=True)

In [44]:
X

,month,n_seconds_1,n_seconds_2,n_seconds_3,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,...,devicebrand_realme,devicebrand_samsung,devicebrand_xiaomi,fe48_mode_menu2__menu8__menu4,fe48_mode_menu6__menu2__menu1,fe48_mode_menu6__menu2__menu4,fe48_mode_menu6__menu2__menu8,fe48_mode_menu6__menu2__menu9,fe48_mode_menu6__menu8__menu1,fe48_mode_menu9__menu4__menu1
0,10,5245.571,981.182,205.948,-1.197737,1.113360,-1.123334,-0.263580,2.161242,2.651375,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,10,2377.463,159.972,79.389,-2.468150,2.103521,0.581307,-1.543583,2.980445,3.309027,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,10,2238.522,582.326,381.049,-1.892554,2.184478,-0.437031,0.512216,3.442060,2.697956,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,10,2105.254,499.524,473.613,-2.204880,1.411003,-0.890840,-0.544155,3.552417,1.872555,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,10,2020.056,691.664,98.314,-2.508528,1.893257,-1.004631,-0.346158,4.002014,1.898146,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94044,11,153.347,91.448,51.669,-1.594276,2.500812,0.441950,-0.078175,3.322438,2.498835,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
94045,11,79.082,75.663,52.589,-1.804187,0.880875,1.017336,-1.055937,4.019820,1.749936,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
94046,12,255.390,124.988,87.040,-0.842930,2.083522,0.302392,-0.713092,1.579907,2.685863,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
94047,12,1061.523,547.669,302.308,-2.454424,2.156997,2.197324,-0.860605,3.877776,1.819750,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [29]:
import lightgbm as lgb
lgb_model = lgb.LGBMClassifier(objective='multiclass', num_class=112)
lgb_model.fit(X,y_encoded)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.048835 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13308
[LightGBM] [Info] Number of data points in the train set: 94049, number of used features: 75
[LightGBM] [Info] Start training from score -6.742041
[LightGBM] [Info] Start training from score -4.758248
[LightGBM] [Info] Start training from score -4.966936
[LightGBM] [Info] Start training from score -5.071449
[LightGBM] [Info] Start training from score -4.514257
[LightGBM] [Info] Start training from score -3.192890
[LightGBM] [Info] Start training from score -3.732441
[LightGBM] [Info] Start training from score -4.861270
[LightGBM] [Info] Start training from score -3.701818
[LightGBM] [Info] Start training from score -4.088292
[LightGBM] [Info] Start training from score -6.639387
[LightGBM] [Info] Start training from score -7.985835
[LightGBM] [Info] Start training from score -7.462587
[LightGB

LGBMClassifier(num_class=112, objective='multiclass')

# Test

In [35]:
model = xgb_model

In [36]:
id_test = df_test['id']
df_test = df_test.drop(['id'],axis=1)

In [37]:
y_pred = model.predict(df_test)
y_pred = encoder.inverse_transform(y_pred)
y_pred

array(['010110000', '000101001', '010010010', ..., '010101000',
       '010100010', '010101000'], dtype=object)

In [41]:
df_train.columns

Index(['id', 'month', 'n_seconds_1', 'n_seconds_2', 'n_seconds_3', 'feature_0',
       'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5',
       'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10',
       'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15',
       'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20',
       'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25',
       'feature_26', 'feature_27', 'feature_28', 'feature_29', 'feature_30',
       'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35',
       'feature_36', 'feature_37', 'feature_38', 'feature_39', 'feature_40',
       'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_45',
       'feature_46', 'feature_47', 'feature_48', 'feature_49', 'target',
       'corr_48_49', 'encoded_target', 'carrier_HAYATEVESIGAR',
       'carrier_KKTCELL', 'carrier_PTTCELL', 'carrier_TEKNOSACELL',
       'carrier_TURKCELL', 'car

In [39]:
pf_sub = ParquetFile("submission_sample_final.parquet")
df_sub = pf_sub.to_pandas()
df_sub.target = y_pred
df_sub.to_csv('sub_xgb_encoded.csv',index=False)

In [33]:
lists = []
for pred in y_pred:
    temp_list=[]
    for i in range(0,len(pred),4):
        decimal_num = int(pred[i:i+4], 2)
        input = 'menu' + str(decimal_num)
        temp_list.append(input)
    lists.append(temp_list)

In [34]:
ikili_veriler = []
for satir in lists:
    ikili_dize = ""
    for menu in range(1, 10):  # Menu numaralarını 1'den 8'e kadar alın
        if f'menu{menu}' in satir:
            ikili_dize += "1"
        else:
            ikili_dize += "0"
    ikili_veriler.append(ikili_dize)

In [36]:
pf_sub = ParquetFile("submission_sample_final.parquet")
df_sub = pf_sub.to_pandas()
df_sub.target = ikili_veriler
df_sub.to_csv('sub_class_lgbm.csv',index=False)